# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [ ]:
# Imports
import os
if('notebooks' in os.getcwd()):
    os.chdir('..')
import json
import pandas as pd
from utils.enums import AirFiles, CO2Files
from utils.load_data import load_data_with_event_matching
from utils.timestamps import read_unix, read_j2000, j2000_to_unix, biopac_file_name_to_unix, generate_biopac_unix_timestamps
import numpy as np


In [ ]:
# Open file index JSON for reading
file_index = pd.read_json(os.path.join(os.getcwd(), 'temp/file_index.json'))
file_index = file_index.sort_index()

In [ ]:
# Example how to retrieve file names for a participant
participant_to_retrieve = 2
air_mask_file = file_index[0][participant_to_retrieve-1][AirFiles.MASK.value]
air_event_file = file_index[0][participant_to_retrieve-1][AirFiles.EVENT.value]

In [ ]:
participant_df = pd.DataFrame()
if os.path.exists(air_mask_file):
    if os.path.exists(air_mask_file):
        print('Loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
        participant_df = load_data_with_event_matching(air_mask_file, True, air_event_file)
        print('Finished loading data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
        participant_df
    else:
        print("Event file missing for participant: " + str(participant_to_retrieve))
else:
    print("Mask file missing for participant: " + str(participant_to_retrieve))





In [ ]:
# Load eye tracking data
participant_eye_df = pd.DataFrame()
air_eyetracking_file = file_index[0][participant_to_retrieve-1][AirFiles.EYE.value]

if os.path.exists(air_eyetracking_file):
    print('Loading eye data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
    participant_eye_df = pd.read_csv(air_eyetracking_file)
    print('Finished loading eye data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
    # First couple frames of eye tracking data are weird. Drop them.
    participant_eye_df = participant_eye_df.drop(index=range(10)).reset_index(drop=True)
else:
    print("Eye file missing for participant: " + str(participant_to_retrieve))

In [ ]:
# Add eye tracking data

# Potential improvements: 
# 1. ignore first 10 eye tracking values as they are a bit weird - DONE
# 2. remove Magnetometer columns

if participant_df.empty or participant_eye_df.empty:
    print('Cant synchronise eye tracking data. One of the files is empty')
else:
    #Some eye tracking files (participants 1-9) have empty 'TimestampUnix column'. Use their 'TimestampJ2000' to calculate unix column
    if np.mean(participant_eye_df['TimestampUnix'].values)==0:
        print('TimestampUnix column for eye data is empty. Converting J2000 timestamps to Unix')
        j2000_timestamps = participant_eye_df['TimestampJ2000']
        unix_timestamps = []
        for j2000_timestamp in j2000_timestamps:
            unix_timestamps.append(j2000_to_unix(j2000_timestamp))
        participant_eye_df['TimestampUnix']=unix_timestamps
        
    participant_eye_df['TimestampUnix'] = participant_eye_df['TimestampUnix']/1000
        
    # TODO: SYNCHRONISE EYE DATA WITH MASK DF
    print('Synchronising mask and eye data files.')
    eye_tracking_columns = ['VerboseData.Right.PupilDiameterMm', 'VerboseData.Left.PupilDiameterMm']
    synced_eye_df = pd.DataFrame(np.nan, index=participant_df.index, columns=eye_tracking_columns)
    for index, eye_data_row in participant_eye_df.iterrows():
        timestamp_to_find = eye_data_row['TimestampUnix']
        event_row_index = participant_df['unix_timestamp'].searchsorted(timestamp_to_find)
        synced_eye_df.loc[event_row_index] = eye_data_row[eye_tracking_columns]
    # eye data file sometimes runs for a few seconds longer than the mask data. 
    # Sync function above contiounsly overwrites last row of data with those eye data frames. Set the last row back to 0 for forward propagation later.     
    synced_eye_df.loc[len(synced_eye_df)-1]=np.nan
    participant_df[eye_tracking_columns] = synced_eye_df
    print('Forward filling eye tracking data')
    participant_df[eye_tracking_columns] = participant_df[eye_tracking_columns].ffill()
    print('Finished synchronising eye tracking with mask data')
    

In [ ]:
# Add biopac data
air_biopac_file = file_index[0][participant_to_retrieve-1][AirFiles.BIOPAC.value]
biopac_start_unix = file_index[0][participant_to_retrieve-1][AirFiles.BIOPAC_UNIX_START_TIME.value]
biopac_column_names = ['Biopac_GSR', 'Biopac_RSP']

if os.path.exists(air_biopac_file) and biopac_start_unix:
    print('Loading biopac data for participant: ' + str(file_index[0][participant_to_retrieve-1]['id']))
    participant_biopac_df = pd.read_csv(air_biopac_file, names=biopac_column_names, index_col=False)
    # Generate biopac unix timestamps at 1000hz from the start of the file
    biopac_unix_timestamps = generate_biopac_unix_timestamps(participant_biopac_df, biopac_start_unix)
    participant_biopac_df = pd.concat([participant_biopac_df,biopac_unix_timestamps], axis=1)
    print('Synchronising biopac data with mask data')
    
    participant_df[biopac_column_names] = np.nan
    if(biopac_start_unix>participant_df.loc[0]['unix_timestamp']):
        print('Mask data started recording first')
        #Mask data started recording first
        #Find first matching frame of biopac data in mask
        first_row_index = participant_df['unix_timestamp'].searchsorted(participant_biopac_df.loc[0]['unix_timestamp'])
        # Since mask data started first, first n of rows in mask data will not have biopac data. Fill them with nans
        nan_rows = pd.DataFrame(np.nan, index=np.arange(first_row_index), columns=biopac_column_names)
        participant_biopac_df = pd.concat([nan_rows, participant_biopac_df]).reset_index(drop=True)
    else:
        print('Biopac data started recoridng first')
        #Biopac data started recording first
        #Find first matching frame of mask data in biopac
        first_row_index = participant_biopac_df['unix_timestamp'].searchsorted(participant_df.loc[0]['unix_timestamp'])
        #Since biopac started first, cant sync first n number of rows. Based on first shared timestamp, drop biopac data before mask recording started
        participant_biopac_df = participant_biopac_df.loc[first_row_index:].reset_index(drop=True)
        #now that both biopac and mask df start at the same time, trim biopac data to end when mask data ends based on the same frequency
        participant_biopac_df = participant_biopac_df.loc[:len(participant_df)-1].reset_index(drop=True)
    
    # merge biopac and mask df
    participant_df[biopac_column_names] = participant_biopac_df[biopac_column_names]

In [ ]:
# Convert Unix
print(read_unix(1651321905.55484))

In [ ]:
# Convert J2000
print(read_j2000(704397173515))